In [1]:
import sys
sys.path.append('./pymlearn/')

In [2]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
import pymlearn.linear
import numpy as np

In [3]:
data_set = datasets.load_digits(10)
test_set_size = data_set['target'].shape[0] / 3
Ytr = data_set['target'][test_set_size:]
Xtr = data_set['data'][test_set_size:]
validation_set_size = Xtr.shape[0] / 3
# Xvd = Xtr[:validation_set_size]
# Yvd = Ytr[:validation_set_size]
# Xtr = Xtr[validation_set_size:]
# Ytr = Ytr[validation_set_size:]
Yts = data_set['target'][:test_set_size]
Xts = data_set['data'][:test_set_size]

In [4]:
p_lambda = 1
W = pymlearn.linear.train_classifer(Xtr, Ytr, loss='softmax', reg=p_lambda)
print 'Accuracy on train: %0.2f%%' % (accuracy_score(Ytr, pymlearn.linear.classify(W, Xtr)) * 100)
print 'Accuracy on test: %0.2f%%' % (accuracy_score(Yts, pymlearn.linear.classify(W, Xts)) * 100)

Accuracy on train: 95.99%
Accuracy on test: 91.65%


In [5]:
p_lambda = 0.1
clf = pymlearn.linear.LinearClassifer(reg=p_lambda, loss='softmax')
clf.fit(Xtr, Ytr)
print 'Accuracy on train: %0.2f%%' % (accuracy_score(Ytr, clf.predict(Xtr)) * 100)
print 'Accuracy on test: %0.2f%%' % (accuracy_score(Yts, clf.predict(Xts)) * 100)

Accuracy on train: 98.91%
Accuracy on test: 92.82%


In [6]:
import sklearn.grid_search

params_grid = {'loss': ['hinge', 'softmax'], 'penalty': ['L1', 'L2'], 
                                                 'reg': np.logspace(-2,2,10)}
estimator = pymlearn.linear.LinearClassifer()
clf = sklearn.grid_search.GridSearchCV(estimator, params_grid, cv=[(np.arange(validation_set_size, Xtr.shape[0]), 
                                                                    np.arange(validation_set_size))], n_jobs=4)
clf.fit(Xtr, Ytr)

print 'Accuracy on train: %0.2f%%' % (accuracy_score(Ytr, clf.predict(Xtr)) * 100)
print 'Accuracy on test: %0.2f%%' % (accuracy_score(Yts, clf.predict(Xts)) * 100)

Accuracy on train: 98.00%
Accuracy on test: 93.49%


In [7]:
clf.best_score_

0.91729323308270672

In [8]:
clf.best_params_

{'loss': 'hinge', 'penalty': 'L2', 'reg': 0.21544346900318834}

In [9]:
%pylab
from scipy.interpolate import interp1d

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [10]:
for i in range(W.shape[0]):
    ax = plt.subplot(2, 5, i+1)
    v = W[i,:-1].reshape((8,8))
    ax.matshow(v)

In [483]:
W_s = W.copy()
W_s[W_s < 1e-2] = 0
W_orig = W.copy()
W = W_s